In [1]:
import os

# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

USER = os.environ['DHUS_USER']
PASSWORD = os.environ['DHUS_PASSWORD']
URL = os.environ['DHUS_URL'] # 'https://scihub.copernicus.eu/dhus'


api = SentinelAPI(USER, PASSWORD, URL)

In [2]:
!cat map.geojson

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -121.453857421875,
              39.81486542536203
            ],
            [
              -121.3336944580078,
              39.81486542536203
            ],
            [
              -121.3336944580078,
              39.90288884886166
            ],
            [
              -121.453857421875,
              39.90288884886166
            ],
            [
              -121.453857421875,
              39.81486542536203
            ]
          ]
        ]
      }
    }
  ]
}


In [3]:
# https://github.com/jupyterlab/jupyter-renderers/tree/master/packages/geojson-extension
from IPython.display import GeoJSON
GeoJSON(data="map.geojson",
        url_template="http://s3-eu-west-1.amazonaws.com/whereonmars.cartodb.net/{basemap_id}/{z}/{x}/{y}.png",
  layer_options={
      "basemap_id": "celestia_mars-shaded-16k_global",
      "attribution" : "Celestia/praesepe",
      "tms": True,
      "minZoom" : 0,
      "maxZoom" : 5
  })

<IPython.display.GeoJSON object>

In [4]:
# https://www.twilio.com/blog/2017/08/geospatial-analysis-python-geojson-geopandas.html
import geopandas as gpd
states = gpd.read_file('map.geojson')
import geojsonio
geojsonio.display(states)

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22id%22%3A%20%220%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%7D%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-121.453857421875%2C%2039.81486542536203%5D%2C%20%5B-121.3336944580078%2C%2039.81486542536203%5D%2C%20%5B-121.3336944580078%2C%2039.90288884886166%5D%2C%20%5B-121.453857421875%2C%2039.90288884886166%5D%2C%20%5B-121.453857421875%2C%2039.81486542536203%5D%5D%5D%7D%2C%20%22bbox%22%3A%20%5B-121.453857421875%2C%2039.81486542536203%2C%20-121.3336944580078%2C%2039.90288884886166%5D%7D%5D%2C%20%22bbox%22%3A%20%5B-121.453857421875%2C%2039.81486542536203%2C%20-121.3336944580078%2C%2039.90288884886166%5D%7D%7D%5D%7D'

In [5]:
footprint = geojson_to_wkt(read_geojson('map.geojson'))
products = api.query(footprint,
                     date=('20181110', date(2018, 11, 12)),
                    #  producttype='SLC',
                     platformname='Sentinel-2',
                    #  orbitdirection='ASCENDING',
                    #  cloudcoverpercentage=(0, 50)
                     )

In [6]:
type(products)

collections.OrderedDict

In [7]:
products

OrderedDict([('313737ac-a9a5-49d0-ae06-280d1856e984',
              {'title': 'S2A_MSIL1C_20181111T185621_N0207_R113_T10TFK_20181111T205747',
               'link': "https://scihub.copernicus.eu/dhus/odata/v1/Products('313737ac-a9a5-49d0-ae06-280d1856e984')/$value",
               'link_alternative': "https://scihub.copernicus.eu/dhus/odata/v1/Products('313737ac-a9a5-49d0-ae06-280d1856e984')/",
               'link_icon': "https://scihub.copernicus.eu/dhus/odata/v1/Products('313737ac-a9a5-49d0-ae06-280d1856e984')/Products('Quicklook')/$value",
               'summary': 'Date: 2018-11-11T18:56:21.024Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 810.36 MB',
               'datatakesensingstart': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
               'beginposition': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
               'endposition': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
               'ingestiondate': datetime.datetime(2018, 11, 11, 22, 

In [8]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

In [9]:
products_df.shape

(1, 32)

In [10]:
products_df.iloc[0]

title                       S2A_MSIL1C_20181111T185621_N0207_R113_T10TFK_2...
link                        https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_alternative            https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_icon                   https://scihub.copernicus.eu/dhus/odata/v1/Pro...
summary                     Date: 2018-11-11T18:56:21.024Z, Instrument: MS...
datatakesensingstart                               2018-11-11 18:56:21.024000
beginposition                                      2018-11-11 18:56:21.024000
endposition                                        2018-11-11 18:56:21.024000
ingestiondate                                      2018-11-11 22:37:47.339000
orbitnumber                                                             17699
relativeorbitnumber                                                       113
cloudcoverpercentage                                                   2.0589
sensoroperationalmode                                           

In [11]:
# sort and limit to first 5 sorted products
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(5)

> * query() for OpenSearch (Solr), which supports filtering products by their attributes and returns metadata for all matched products at once.
> * get_product_odata() for OData, which can be queried one product at a time but provides the full metadata available for each product, as well as information about the product file such as the file size and checksum, which are not available from OpenSearch.

In [12]:
# Get basic information about the product: its title, file size, MD5 sum, date, footprint and
# its download url
product_id = products_df_sorted['uuid'].values[0]
basic_infos = api.get_product_odata(product_id)
basic_infos

{'id': '313737ac-a9a5-49d0-ae06-280d1856e984',
 'title': 'S2A_MSIL1C_20181111T185621_N0207_R113_T10TFK_20181111T205747',
 'size': 849764964,
 'md5': 'C76F928E3766081361A3502F044AFF2E',
 'date': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
 'footprint': 'POLYGON((-121.81731 40.64479964997401,-120.51955 40.62420990912828,-120.55524 39.63587147039632,-121.834335 39.65575725270223,-121.81731 40.64479964997401))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('313737ac-a9a5-49d0-ae06-280d1856e984')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2018, 11, 11, 22, 38, 44, 731000),
 'Ingestion Date': datetime.datetime(2018, 11, 11, 22, 37, 47, 339000)}

In [13]:
product_id = products_df_sorted['uuid'].values[0]

# Get the product's full metadata available on the server
full_infos = api.get_product_odata(product_id, full=True)
full_infos

{'id': '313737ac-a9a5-49d0-ae06-280d1856e984',
 'title': 'S2A_MSIL1C_20181111T185621_N0207_R113_T10TFK_20181111T205747',
 'size': 849764964,
 'md5': 'C76F928E3766081361A3502F044AFF2E',
 'date': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
 'footprint': 'POLYGON((-121.81731 40.64479964997401,-120.51955 40.62420990912828,-120.55524 39.63587147039632,-121.834335 39.65575725270223,-121.81731 40.64479964997401))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('313737ac-a9a5-49d0-ae06-280d1856e984')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2018, 11, 11, 22, 38, 44, 731000),
 'Ingestion Date': datetime.datetime(2018, 11, 11, 22, 37, 47, 339000),
 'Cloud cover percentage': 2.0589,
 'Datatake sensing start': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
 'Date': datetime.datetime(2018, 11, 11, 18, 56, 21, 24000),
 'Degraded ancillary data percentage': 0.0,
 'Degraded MSI data percentage': 0,
 'Filename': 'S2A_MSIL1C_20181111T185621_N0207_R113_T10